In [22]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import gc

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

#cats
from catboost import CatBoostClassifier, Pool, cv

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn import preprocessing
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


## Load Train and Test Data

In [3]:
# Read train and test files
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 201 entries, ID_code to var_199
dtypes: float64(200), object(1)
memory usage: 306.7+ MB


## Check for Missing Values

In [8]:
#### Check if there are any NULL values in Train Data
print("Total Train Features with NaN Values = " + str(train_df.columns[train_df.isnull().sum() != 0].size))
if (train_df.columns[train_df.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(train_df.columns[train_df.isnull().sum() != 0])))
    train_df[train_df.columns[train_df.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Train Features with NaN Values = 0


In [9]:
#### Check if there are any NULL values in Test Data
print("Total Test Features with NaN Values = " + str(test_df.columns[test_df.isnull().sum() != 0].size))
if (test_df.columns[test_df.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(test_df.columns[test_df.isnull().sum() != 0])))
    test_df[test_df.columns[test_df.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Test Features with NaN Values = 0


## Check and Remove Constant Features

In [14]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [15]:
# check and remove constant columns
colsToRemove = []
for col in train_df.columns:
    if col != 'ID_code' and col != 'target':
        if train_df[col].std() == 0: 
            colsToRemove.append(col)
        
# remove constant columns in the training set
train_df.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
test_df.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `0` Constant Columns

[]


In [16]:
%%time
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            ia = vs.iloc[:,i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:,j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
                    break

    return dups

colsToRemove = duplicate_columns(train_df)
print(colsToRemove)

[]
Wall time: 6.84 s


In [19]:
#remove duplicate columns in the training set
train_df.drop(colsToRemove, axis=1, inplace=True) 

#remove duplicate columns in the testing set
test_df.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `0` Duplicate Columns

[]


## Drop Sparse Data

In [20]:
def drop_sparse(train, test):
    flist = [x for x in train.columns if not x in ['ID_code','target']]
    for f in flist:
        if len(np.unique(train[f]))<2:
            train.drop(f, axis=1, inplace=True)
            test.drop(f, axis=1, inplace=True)
    return train, test

In [21]:
%%time
train_df, test_df = drop_sparse(train_df, test_df)

Wall time: 4.01 s


In [41]:
train_df.fillna(999, inplace = True)
test_df.fillna(999, inplace = True)

In [42]:
gc.collect()
print("Train set size: {}".format(train_df.shape))
print("Test set size: {}".format(test_df.shape))

Train set size: (200000, 202)
Test set size: (200000, 201)


# Scaling

In [43]:
train_df.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [44]:
from sklearn import preprocessing

In [45]:
y = train_df['target']
X = train_df.drop(['ID_code', 'target'], axis=1)

X_test = test_df.drop(['ID_code'], axis=1)
X_test.fillna(999, inplace = True)
object_columns_test = X_test.select_dtypes(include=['object']).columns.values.tolist()

In [46]:
scaler = preprocessing.StandardScaler().fit(X)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [47]:
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

## MinMaX scaling

In [58]:
 from sklearn.preprocessing import MinMaxScaler

In [62]:
#Normalise the dataset. This is required for getting the variance threshold
scaler = MinMaxScaler().fit(X)
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [63]:
X_minmax = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

In [69]:
X_scaled.std()

1.0000000000000013

## CatBoost

In [48]:
X_train, X_validation, y_train, y_validation = train_test_split(X_scaled, y, train_size=0.8,  random_state=1234)

C:\Users\a.moiseev\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [49]:
model = CatBoostClassifier(
    eval_metric='AUC'
    ,use_best_model=True
    ,random_seed=42
    ,rsm = 0.8
    ,scale_pos_weight = 9
    ,learning_rate = 0.01
    ,l2_leaf_reg = 10000
    ,od_type = 'Iter'
    ,od_pval = 0.001
    ,iterations= 10000
                         )

In [50]:
model.fit(X_train, y_train, object_columns_test, eval_set=(X_validation, y_validation), plot=True, use_best_model = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6358697	best: 0.6358697 (0)	total: 292ms	remaining: 48m 39s
1:	test: 0.6453164	best: 0.6453164 (1)	total: 550ms	remaining: 45m 50s
2:	test: 0.6459603	best: 0.6459603 (2)	total: 867ms	remaining: 48m 7s
3:	test: 0.6552978	best: 0.6552978 (3)	total: 1.17s	remaining: 48m 37s
4:	test: 0.6602012	best: 0.6602012 (4)	total: 1.54s	remaining: 51m 14s
5:	test: 0.6619381	best: 0.6619381 (5)	total: 1.86s	remaining: 51m 41s
6:	test: 0.6634299	best: 0.6634299 (6)	total: 2.19s	remaining: 52m 5s
7:	test: 0.6693496	best: 0.6693496 (7)	total: 2.5s	remaining: 52m 2s
8:	test: 0.6721303	best: 0.6721303 (8)	total: 2.84s	remaining: 52m 37s
9:	test: 0.6734170	best: 0.6734170 (9)	total: 3.26s	remaining: 54m 17s
10:	test: 0.6751771	best: 0.6751771 (10)	total: 3.56s	remaining: 53m 53s
11:	test: 0.6751394	best: 0.6751771 (10)	total: 3.87s	remaining: 53m 38s
12:	test: 0.6777224	best: 0.6777224 (12)	total: 4.23s	remaining: 54m 13s
13:	test: 0.6774238	best: 0.6777224 (12)	total: 4.55s	remaining: 54m 3s
14:

113:	test: 0.7390050	best: 0.7390050 (113)	total: 36.7s	remaining: 53m
114:	test: 0.7389990	best: 0.7390050 (113)	total: 36.9s	remaining: 52m 54s
115:	test: 0.7397673	best: 0.7397673 (115)	total: 37.2s	remaining: 52m 51s
116:	test: 0.7401380	best: 0.7401380 (116)	total: 37.5s	remaining: 52m 46s
117:	test: 0.7405560	best: 0.7405560 (117)	total: 37.8s	remaining: 52m 43s
118:	test: 0.7407924	best: 0.7407924 (118)	total: 38s	remaining: 52m 38s
119:	test: 0.7412158	best: 0.7412158 (119)	total: 38.3s	remaining: 52m 34s
120:	test: 0.7414220	best: 0.7414220 (120)	total: 38.6s	remaining: 52m 31s
121:	test: 0.7413297	best: 0.7414220 (120)	total: 38.9s	remaining: 52m 26s
122:	test: 0.7418132	best: 0.7418132 (122)	total: 39.1s	remaining: 52m 23s
123:	test: 0.7423696	best: 0.7423696 (123)	total: 39.4s	remaining: 52m 19s
124:	test: 0.7428360	best: 0.7428360 (124)	total: 39.7s	remaining: 52m 16s
125:	test: 0.7432703	best: 0.7432703 (125)	total: 40s	remaining: 52m 12s
126:	test: 0.7438770	best: 0.7438

223:	test: 0.7780901	best: 0.7780901 (223)	total: 1m 10s	remaining: 51m 35s
224:	test: 0.7780025	best: 0.7780901 (223)	total: 1m 11s	remaining: 51m 35s
225:	test: 0.7782925	best: 0.7782925 (225)	total: 1m 11s	remaining: 51m 32s
226:	test: 0.7783942	best: 0.7783942 (226)	total: 1m 11s	remaining: 51m 33s
227:	test: 0.7779464	best: 0.7783942 (226)	total: 1m 12s	remaining: 51m 36s
228:	test: 0.7777593	best: 0.7783942 (226)	total: 1m 12s	remaining: 51m 36s
229:	test: 0.7777820	best: 0.7783942 (226)	total: 1m 12s	remaining: 51m 38s
230:	test: 0.7778547	best: 0.7783942 (226)	total: 1m 13s	remaining: 51m 39s
231:	test: 0.7781101	best: 0.7783942 (226)	total: 1m 13s	remaining: 51m 39s
232:	test: 0.7782300	best: 0.7783942 (226)	total: 1m 13s	remaining: 51m 37s
233:	test: 0.7786266	best: 0.7786266 (233)	total: 1m 14s	remaining: 51m 36s
234:	test: 0.7788237	best: 0.7788237 (234)	total: 1m 14s	remaining: 51m 35s
235:	test: 0.7792878	best: 0.7792878 (235)	total: 1m 14s	remaining: 51m 32s
236:	test: 0

331:	test: 0.7974343	best: 0.7974343 (331)	total: 1m 45s	remaining: 51m 17s
332:	test: 0.7976772	best: 0.7976772 (332)	total: 1m 45s	remaining: 51m 16s
333:	test: 0.7977988	best: 0.7977988 (333)	total: 1m 46s	remaining: 51m 16s
334:	test: 0.7980363	best: 0.7980363 (334)	total: 1m 46s	remaining: 51m 14s
335:	test: 0.7982571	best: 0.7982571 (335)	total: 1m 46s	remaining: 51m 13s
336:	test: 0.7984019	best: 0.7984019 (336)	total: 1m 47s	remaining: 51m 13s
337:	test: 0.7984768	best: 0.7984768 (337)	total: 1m 47s	remaining: 51m 13s
338:	test: 0.7987710	best: 0.7987710 (338)	total: 1m 47s	remaining: 51m 12s
339:	test: 0.7987755	best: 0.7987755 (339)	total: 1m 48s	remaining: 51m 11s
340:	test: 0.7989998	best: 0.7989998 (340)	total: 1m 48s	remaining: 51m 12s
341:	test: 0.7991873	best: 0.7991873 (341)	total: 1m 48s	remaining: 51m 10s
342:	test: 0.7991053	best: 0.7991873 (341)	total: 1m 49s	remaining: 51m 9s
343:	test: 0.7992937	best: 0.7992937 (343)	total: 1m 49s	remaining: 51m 9s
344:	test: 0.7

...

9508:	test: 0.8984966	best: 0.8984969 (9503)	total: 56m 9s	remaining: 2m 54s
9509:	test: 0.8984978	best: 0.8984978 (9509)	total: 56m 10s	remaining: 2m 53s
9510:	test: 0.8984984	best: 0.8984984 (9510)	total: 56m 11s	remaining: 2m 53s
9511:	test: 0.8984980	best: 0.8984984 (9510)	total: 56m 11s	remaining: 2m 52s
9512:	test: 0.8984981	best: 0.8984984 (9510)	total: 56m 11s	remaining: 2m 52s
9513:	test: 0.8985010	best: 0.8985010 (9513)	total: 56m 12s	remaining: 2m 52s
9514:	test: 0.8985029	best: 0.8985029 (9514)	total: 56m 12s	remaining: 2m 51s
9515:	test: 0.8985045	best: 0.8985045 (9515)	total: 56m 13s	remaining: 2m 51s
9516:	test: 0.8985072	best: 0.8985072 (9516)	total: 56m 13s	remaining: 2m 51s
9517:	test: 0.8985058	best: 0.8985072 (9516)	total: 56m 14s	remaining: 2m 50s
9518:	test: 0.8985046	best: 0.8985072 (9516)	total: 56m 14s	remaining: 2m 50s
9519:	test: 0.8985062	best: 0.8985072 (9516)	total: 56m 15s	remaining: 2m 50s
9520:	test: 0.8985064	best: 0.8985072 (9516)	total: 56m 15s	remai

9614:	test: 0.8985946	best: 0.8985950 (9611)	total: 56m 51s	remaining: 2m 16s
9615:	test: 0.8985935	best: 0.8985950 (9611)	total: 56m 52s	remaining: 2m 16s
9616:	test: 0.8985929	best: 0.8985950 (9611)	total: 56m 52s	remaining: 2m 15s
9617:	test: 0.8985905	best: 0.8985950 (9611)	total: 56m 52s	remaining: 2m 15s
9618:	test: 0.8985903	best: 0.8985950 (9611)	total: 56m 53s	remaining: 2m 15s
9619:	test: 0.8985915	best: 0.8985950 (9611)	total: 56m 53s	remaining: 2m 14s
9620:	test: 0.8985935	best: 0.8985950 (9611)	total: 56m 53s	remaining: 2m 14s
9621:	test: 0.8985924	best: 0.8985950 (9611)	total: 56m 54s	remaining: 2m 14s
9622:	test: 0.8985920	best: 0.8985950 (9611)	total: 56m 54s	remaining: 2m 13s
9623:	test: 0.8985931	best: 0.8985950 (9611)	total: 56m 55s	remaining: 2m 13s
9624:	test: 0.8985915	best: 0.8985950 (9611)	total: 56m 55s	remaining: 2m 13s
9625:	test: 0.8985933	best: 0.8985950 (9611)	total: 56m 56s	remaining: 2m 12s
9626:	test: 0.8985940	best: 0.8985950 (9611)	total: 56m 56s	rema

9720:	test: 0.8986748	best: 0.8986759 (9718)	total: 57m 32s	remaining: 1m 39s
9721:	test: 0.8986752	best: 0.8986759 (9718)	total: 57m 32s	remaining: 1m 38s
9722:	test: 0.8986785	best: 0.8986785 (9722)	total: 57m 33s	remaining: 1m 38s
9723:	test: 0.8986810	best: 0.8986810 (9723)	total: 57m 33s	remaining: 1m 38s
9724:	test: 0.8986826	best: 0.8986826 (9724)	total: 57m 33s	remaining: 1m 37s
9725:	test: 0.8986815	best: 0.8986826 (9724)	total: 57m 34s	remaining: 1m 37s
9726:	test: 0.8986820	best: 0.8986826 (9724)	total: 57m 34s	remaining: 1m 36s
9727:	test: 0.8986818	best: 0.8986826 (9724)	total: 57m 34s	remaining: 1m 36s
9728:	test: 0.8986831	best: 0.8986831 (9728)	total: 57m 35s	remaining: 1m 36s
9729:	test: 0.8986836	best: 0.8986836 (9729)	total: 57m 35s	remaining: 1m 35s
9730:	test: 0.8986843	best: 0.8986843 (9730)	total: 57m 35s	remaining: 1m 35s
9731:	test: 0.8986833	best: 0.8986843 (9730)	total: 57m 36s	remaining: 1m 35s
9732:	test: 0.8986838	best: 0.8986843 (9730)	total: 57m 36s	rema

9826:	test: 0.8987491	best: 0.8987506 (9825)	total: 58m 9s	remaining: 1m 1s
9827:	test: 0.8987490	best: 0.8987506 (9825)	total: 58m 10s	remaining: 1m 1s
9828:	test: 0.8987496	best: 0.8987506 (9825)	total: 58m 10s	remaining: 1m
9829:	test: 0.8987499	best: 0.8987506 (9825)	total: 58m 10s	remaining: 1m
9830:	test: 0.8987508	best: 0.8987508 (9830)	total: 58m 10s	remaining: 1m
9831:	test: 0.8987489	best: 0.8987508 (9830)	total: 58m 11s	remaining: 59.7s
9832:	test: 0.8987509	best: 0.8987509 (9832)	total: 58m 12s	remaining: 59.3s
9833:	test: 0.8987561	best: 0.8987561 (9833)	total: 58m 12s	remaining: 59s
9834:	test: 0.8987572	best: 0.8987572 (9834)	total: 58m 12s	remaining: 58.6s
9835:	test: 0.8987573	best: 0.8987573 (9835)	total: 58m 13s	remaining: 58.2s
9836:	test: 0.8987613	best: 0.8987613 (9836)	total: 58m 13s	remaining: 57.9s
9837:	test: 0.8987618	best: 0.8987618 (9837)	total: 58m 14s	remaining: 57.5s
9838:	test: 0.8987609	best: 0.8987618 (9837)	total: 58m 14s	remaining: 57.2s
9839:	test:

9933:	test: 0.8988322	best: 0.8988322 (9933)	total: 58m 50s	remaining: 23.5s
9934:	test: 0.8988326	best: 0.8988326 (9934)	total: 58m 50s	remaining: 23.1s
9935:	test: 0.8988345	best: 0.8988345 (9935)	total: 58m 51s	remaining: 22.7s
9936:	test: 0.8988367	best: 0.8988367 (9936)	total: 58m 51s	remaining: 22.4s
9937:	test: 0.8988362	best: 0.8988367 (9936)	total: 58m 52s	remaining: 22s
9938:	test: 0.8988369	best: 0.8988369 (9938)	total: 58m 52s	remaining: 21.7s
9939:	test: 0.8988385	best: 0.8988385 (9939)	total: 58m 52s	remaining: 21.3s
9940:	test: 0.8988385	best: 0.8988385 (9939)	total: 58m 52s	remaining: 21s
9941:	test: 0.8988389	best: 0.8988389 (9941)	total: 58m 53s	remaining: 20.6s
9942:	test: 0.8988388	best: 0.8988389 (9941)	total: 58m 53s	remaining: 20.3s
9943:	test: 0.8988401	best: 0.8988401 (9943)	total: 58m 54s	remaining: 19.9s
9944:	test: 0.8988408	best: 0.8988408 (9944)	total: 58m 54s	remaining: 19.5s
9945:	test: 0.8988420	best: 0.8988420 (9945)	total: 58m 54s	remaining: 19.2s
994

In [51]:
print (round(model.score(X_validation, y_validation),4))
print (round(model.score(X_train, y_train),4))

0.8342
0.8427


In [52]:
params = model.get_params()
params['iterations'] = 2000
params['custom_loss'] = 'AUC'


cv_data = cv(
    params = params,
    pool = Pool(X_scaled, y, object_columns_test),
    fold_count=5,
    inverted=False,
    shuffle=True,
    partition_random_seed=0,
    plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6212507	best: 0.6212507 (0)
1:	test: 0.6418544	best: 0.6418544 (1)
2:	test: 0.6435371	best: 0.6435371 (2)
3:	test: 0.6476826	best: 0.6476826 (3)
4:	test: 0.6510717	best: 0.6510717 (4)
5:	test: 0.6560045	best: 0.6560045 (5)
6:	test: 0.6596216	best: 0.6596216 (6)
7:	test: 0.6631576	best: 0.6631576 (7)
8:	test: 0.6633863	best: 0.6633863 (8)
9:	test: 0.6668024	best: 0.6668024 (9)
10:	test: 0.6686938	best: 0.6686938 (10)
11:	test: 0.6706282	best: 0.6706282 (11)
12:	test: 0.6710882	best: 0.6710882 (12)
13:	test: 0.6738600	best: 0.6738600 (13)
14:	test: 0.6758849	best: 0.6758849 (14)
15:	test: 0.6779102	best: 0.6779102 (15)
16:	test: 0.6775991	best: 0.6779102 (15)
17:	test: 0.6777419	best: 0.6779102 (15)
18:	test: 0.6778398	best: 0.6779102 (15)
19:	test: 0.6790032	best: 0.6790032 (19)
20:	test: 0.6792774	best: 0.6792774 (20)
21:	test: 0.6800335	best: 0.6800335 (21)
22:	test: 0.6803442	best: 0.6803442 (22)
23:	test: 0.6809319	best: 0.6809319 (23)
24:	test: 0.6809200	best: 0.6809319 

971:	test: 0.8379190	best: 0.8379190 (971)
972:	test: 0.8379519	best: 0.8379519 (972)
973:	test: 0.8380211	best: 0.8380211 (973)
974:	test: 0.8380579	best: 0.8380579 (974)
975:	test: 0.8381020	best: 0.8381020 (975)
976:	test: 0.8381186	best: 0.8381186 (976)
977:	test: 0.8381787	best: 0.8381787 (977)
978:	test: 0.8382248	best: 0.8382248 (978)
979:	test: 0.8382807	best: 0.8382807 (979)
980:	test: 0.8383053	best: 0.8383053 (980)
981:	test: 0.8383660	best: 0.8383660 (981)
982:	test: 0.8384246	best: 0.8384246 (982)
983:	test: 0.8384728	best: 0.8384728 (983)
984:	test: 0.8385181	best: 0.8385181 (984)
985:	test: 0.8385715	best: 0.8385715 (985)
986:	test: 0.8386178	best: 0.8386178 (986)
987:	test: 0.8386748	best: 0.8386748 (987)
988:	test: 0.8386932	best: 0.8386932 (988)
989:	test: 0.8387318	best: 0.8387318 (989)
990:	test: 0.8387763	best: 0.8387763 (990)
991:	test: 0.8387921	best: 0.8387921 (991)
992:	test: 0.8387946	best: 0.8387946 (992)
993:	test: 0.8388210	best: 0.8388210 (993)
994:	test: 

1945:	test: 0.8645597	best: 0.8645597 (1945)
1946:	test: 0.8645681	best: 0.8645681 (1946)
1947:	test: 0.8645810	best: 0.8645810 (1947)
1948:	test: 0.8646005	best: 0.8646005 (1948)
1949:	test: 0.8646170	best: 0.8646170 (1949)
1950:	test: 0.8646322	best: 0.8646322 (1950)
1951:	test: 0.8646470	best: 0.8646470 (1951)
1952:	test: 0.8646634	best: 0.8646634 (1952)
1953:	test: 0.8646866	best: 0.8646866 (1953)
1954:	test: 0.8647024	best: 0.8647024 (1954)
1955:	test: 0.8647146	best: 0.8647146 (1955)
1956:	test: 0.8647289	best: 0.8647289 (1956)
1957:	test: 0.8647418	best: 0.8647418 (1957)
1958:	test: 0.8647666	best: 0.8647666 (1958)
1959:	test: 0.8647775	best: 0.8647775 (1959)
1960:	test: 0.8647965	best: 0.8647965 (1960)
1961:	test: 0.8648111	best: 0.8648111 (1961)
1962:	test: 0.8648267	best: 0.8648267 (1962)
1963:	test: 0.8648317	best: 0.8648317 (1963)
1964:	test: 0.8648418	best: 0.8648418 (1964)
1965:	test: 0.8648622	best: 0.8648622 (1965)
1966:	test: 0.8648852	best: 0.8648852 (1966)
1967:	test

In [56]:
print ('AUC mean:', round(cv_data['test-AUC-mean'].max(),8))
print ('AUC mean:', round(cv_data['test-AUC-mean'].mean(),8))
print ('Logloss mean:', round(cv_data['test-Logloss-mean'].mean(), 8))

AUC mean: 0.86537402
AUC mean: 0.82336785
Logloss mean: 0.58118901


In [54]:
preds = model.predict_proba(X_test)

In [55]:
num_sub = '_standart_cat'
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = preds[:,1]
sub.to_csv('submission{}.csv'.format(num_sub), index=False)